In [1]:
import tensorflow as tf
import numpy as np
import config as cfg
import csv
import pandas as pd
from PIL import Image
import cv2
import os

In [28]:
def read_whole_csv(path):
    file = open(path)
    csvreader = csv.reader(file)
    all = []
    next(csvreader, None)
    for row in csvreader:
        ye = True
        for value in row:
            if value.startswith('-'):
                ye = False
                break
        if ye:
            all.append([row[0], [int(row[1]), int(row[2])],
                       int(row[3]) - int(row[1]), row[5:]])
    file.close()
    return all


In [29]:
def split_dataset(dataset, test_ratio=0.20):
    test_indices = np.random.rand(len(dataset)) < test_ratio
    return dataset[~test_indices], dataset[test_indices]


In [30]:
def crop_face(image, x, y, w, h):
    image = image[y:y + h, x:x + w]
    return image


In [31]:
def prep_image(dp):
    wh = int(dp[2])
    points = [(float(a) / wh) for a in dp[3]]
    # crop out face from image
    image = cv2.imread(cfg.IMAGES_PATH + dp[0])
    image = crop_face(image, int(dp[1][0]), int(dp[1][1]), wh, wh)

    # resize image to xxx * xxx
    image = Image.fromarray(image)
    image = image.resize((cfg.CROP_SIZE, cfg.CROP_SIZE))
    image = np.array(image)

    # (should I really convert it to tensor?)
    points = tf.convert_to_tensor(points, dtype=tf.float32)

    # PROFIT?
    return image, points


In [32]:
ds = read_whole_csv(cfg.LABELS_PATH)
ds = pd.DataFrame(ds)
train_ds, test_ds = split_dataset(ds)

In [33]:
# new
trainX = []
trainY = []
testX = []
testY = []

for i in range(train_ds.shape[0]):
    image, points = prep_image(train_ds.iloc[i])
    input_image = tf.cast(image, dtype=tf.int32)
    trainX.append(input_image)
    trainY.append(points)

for i in range(test_ds.shape[0]):
    image, points = prep_image(train_ds.iloc[i])
    input_image = tf.cast(image, dtype=tf.int32)
    testX.append(input_image)
    testY.append(points)

trainX_ = tf.convert_to_tensor(trainX, dtype=tf.float32) / 255
trainY_ = tf.convert_to_tensor(trainY, dtype=tf.float32)

testX_ = tf.convert_to_tensor(testX, dtype=tf.float32) / 255
testY_ = tf.convert_to_tensor(testY, dtype=tf.float32)


In [67]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras import backend as K



def build_old(image_size):
    model = Sequential()
    inputShape = (image_size, image_size, 3)
    # CONV => RELU => POOL
    model.add(Conv2D(32, (3, 3), padding="same",
                     input_shape=inputShape))
    model.add(Activation("relu"))

    model.add(MaxPooling2D(pool_size=(3, 3)))
    model.add(Dropout(0.25))

    # (CONV => RELU) * 2 => POOL
    model.add(Conv2D(64, (3, 3), padding="same"))
    model.add(Activation("relu"))
    # model.add(Conv2D(128, (3, 3), padding="same"))
    # model.add(Activation("relu"))

    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    # (CONV => RELU) * 2 => POOL
    # model.add(Conv2D(32, (3, 3), padding="same"))
    # model.add(Activation("relu"))
    # model.add(Conv2D(128, (3, 3), padding="same"))
    # model.add(Activation("relu"))
    # model.add(Dropout(0.25))
    
    # first (and only) set of FC => RELU layers
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation("relu"))
    # model.add(Dropout(0.5))
    model.add(Dense(cfg.OUTPUTS))
    return model


def build(size):
    model = Sequential()

    # model.add(BatchNormalization(input_shape=(size, size, 3)))

    model.add(Conv2D(24, (5, 5), padding='same',
              input_shape=(size, size, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))

    model.add(Conv2D(36, (5, 5)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2),
              strides=(2, 2), padding='valid'))

    model.add(Conv2D(48, (5, 5)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2),
              strides=(2, 2), padding='valid'))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2),
              strides=(2, 2), padding='valid'))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(GlobalAveragePooling2D())

    model.add(Dense(1000, activation='relu'))
    model.add(Dense(500, activation='relu'))
    model.add(Dense(136))
    return model


In [2]:
def get_model(load_latest = False):
    model = build(cfg.CROP_SIZE)

    # old
    # model.compile(optimizer='adam',
    #             loss=tf.keras.losses.Huber(),
    #             metrics=['accuracy'])

    model.compile(optimizer='rmsprop', loss='mse', metrics=['accuracy'])

    if load_latest:
        checkpoint_dir = os.path.dirname(cfg.CHECKPOINT_PATH)
        latest = tf.train.latest_checkpoint(checkpoint_dir)
        model.load_weights(latest)
       
    return model

def load_model(path):
    model = tf.keras.models.load_model(path)
    return model

def evaluate_model(model, x, y):
    loss, acc = model.evaluate(testX_, testY_, verbose=2)
    print(f'Model, accuracy: {acc*100:5.2f}%')

def save_model(model, path):
    model.save(path)


In [68]:
model = get_model()

In [106]:
batch_size = 78
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=cfg.CHECKPOINT_PATH,
    verbose=1,
    save_weights_only=True,
    save_freq=5*batch_size)

model.fit(trainX_, 
          trainY_,
          epochs=1,
          validation_data=(testX_, testY_),
          shuffle=True,
          callbacks=[cp_callback])



78/78 [==============================] - 43s 547ms/step - loss: 1.2793e-04 - accuracy: 0.8916 - val_loss: 1.1535e-04 - val_accuracy: 0.8938


In [107]:
# save_model(model, 'models/x128NA-89.h5')

In [108]:
evaluate_model(model, testX_, testY_)

20/20 - 3s - loss: 1.1535e-04 - accuracy: 0.8938 - 3s/epoch - 144ms/step
Model, accuracy: 89.38%


In [4]:
def predictor(model, image_path, size):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    faceCascade = cv2.CascadeClassifier(
        cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.3,
        minNeighbors=3,
        minSize=(30, 30)
    )

    # dimsCrop = [[110, 110, 300, 300]]
    dimsCrop = []
    for (x, y, w, h) in faces:
        image = image[y:y + h, x:x + w]
        dimsCrop.append([x,y,w,h])
        
    image = Image.fromarray(image)
    image = image.resize((size, size))
    image = np.array(image)
    
    # predict
    input_image = tf.cast(image, dtype=tf.float32) / 255
    input_image = tf.expand_dims(input_image, axis=0)
    res = model.predict(input_image)
    # print(res)
    x_ = []
    y_ = []
    for i in range(0, 136, 2):
        x_.append(int(res[0][i] * size * (dimsCrop[0][3] / size) + dimsCrop[0][0]))
        y_.append(int(res[0][i+1]* size * (dimsCrop[0][3] / size) + dimsCrop[0][1]))

    image = cv2.imread(image_path)
    for i in range(68):
        image = cv2.circle(image, (x_[i], y_[i]), 1, (0, 0, 255), int(image.shape[1] * 0.006))
    cv2.imwrite('pic.jpeg', image)
    
    

# predictor(model, 'data\data\InitialDS\\0 (13).jpg', 128)
predictor(model, 'data\data\InitialDS\\0 (19).jpg', 128)
# predictor(model, 'ay.JPG', 128)

In [3]:
# model = load_model('models\\x128-92.78.h5')
model = load_model('models\\x128NA-89.h5')


In [101]:
# my implementation
cap = cv2.VideoCapture(0)
while True:
    size = 128
    _, frame = cap.read()
    frame = cv2.copyMakeBorder(frame, 50, 50, 50, 50, cv2.BORDER_CONSTANT)
    gray = cv2.cvtColor(src=frame, code=cv2.COLOR_BGR2GRAY)

    faceCascade = cv2.CascadeClassifier(
        cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.3,
        minNeighbors=3,
        minSize=(30, 30)
    )
    dimsCrop = []
    for (x, y, w, h) in faces:
        image = frame[y:y + h, x:x + w]
        dimsCrop.append([x, y, w, h])
        cv2.rectangle(img=frame, pt1=(x, y), pt2=(
            x+w, y+h), color=(0, 255, 0), thickness=4)

    image = Image.fromarray(image)
    image = image.resize((size, size))
    image = np.array(image)

    # predict
    input_image = tf.cast(image, dtype=tf.float32) / 255
    input_image = tf.expand_dims(input_image, axis=0)
    res = model.predict(input_image)
    x_ = []
    y_ = []
    if len(dimsCrop) > 0:
        for i in range(0, 136, 2):
            x_.append(
                int(res[0][i] * size * (dimsCrop[0][3] / size) + dimsCrop[0][0]))
            y_.append(int(res[0][i+1] * size *
                    (dimsCrop[0][3] / size) + dimsCrop[0][1]))

        for i in range(68):
            image = cv2.circle(frame, (x_[i], y_[i]), 1,
                            (0, 0, 255), int(image.shape[1] * 0.006))
    cv2.imshow(winname="Face", mat=frame)
    if cv2.waitKey(delay=1) == 27:
        break

cap.release()
cv2.destroyAllWindows()


In [24]:
cap.release()
cv2.destroyAllWindows()

In [1]:
# somewhat real (not mine tho)
import cv2
import dlib

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
cap = cv2.VideoCapture(0)

while True:
    _, frame = cap.read()
    gray = cv2.cvtColor(src=frame, code=cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    for face in faces:
        x1 = face.left()  # left point
        y1 = face.top()  # top point
        x2 = face.right()  # right point
        y2 = face.bottom()  # bottom point

        landmarks = predictor(image=gray, box=face)
        for n in range(0, 68):
            x = landmarks.part(n).x
            y = landmarks.part(n).y
            cv2.circle(img=frame, center=(x, y), radius=3,
                       color=(0, 255, 0), thickness=-1)

    cv2.imshow(winname="Face", mat=frame)
    if cv2.waitKey(delay=1) == 27:
        break

cap.release()
cv2.destroyAllWindows()
